<a href="https://colab.research.google.com/github/ShounakDas101/AIML_Hari/blob/main/ML4SCI_BinaryClassificationCNN_histo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import h5py
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init
from torch.utils.data import Dataset, random_split, DataLoader
from torchvision import transforms
import torch.optim as optim

import matplotlib.pyplot as plt

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


In [2]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
DATA_DIR='./data'
'''
IMAGE_SIZE = 32 # W=32 H=32
NUM_CHANNELS = 2 # R G B then it should be 3
PATCH_SIZE = 4 # W_patch=4 H_patch=4
NUM_PATCHES = (IMAGE_SIZE//PATCH_SIZE)*(IMAGE_SIZE//PATCH_SIZE) # = 64
NUM_HEADS = 8  # enbedding dim should be divisible by no of heads
EMBBEDING_DIM = PATCH_SIZE*PATCH_SIZE*NUM_CHANNELS  #NUM_PATCHNELS
MLP_SIZE = 128
NUM_LAYERS = 8
'''
NUM_CLASSES = 2
NUM_WORKERS = 8
BATCH_SIZE = 256
EPOCHS = 40
DROP_OUT = 0.1
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-1

In [5]:
# clearing cuda cache memory
import gc
torch.cuda.empty_cache()
gc.collect()

30

In [ ]:
'''
import os

# Specify the path to the file in your Google Drive
file_path = '/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'

# CHECKPOINT_PATH = '/content/drive/MyDrive/CHECKPOINTS'

# Check if the file exists
if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
else:
    print(f"The file '{file_path}' does not exist.")

'''

In [ ]:
import h5py
'''
def find_keys(hdf5_obj, path='/'):
    """Recursively find keys in an HDF5 file."""
    keys = []
    for key in hdf5_obj[path].keys():
        full_path = f"{path}/{key}"
        keys.append(full_path)
        if isinstance(hdf5_obj[full_path], h5py.Group):
            keys.extend(find_keys(hdf5_obj, full_path))
    return keys

# Open the HDF5 file
file_path = 'SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'  # Replace with your actual file path
with h5py.File(file_path, 'r') as hdf5_file:
    # Find keys starting from the root
    all_keys = find_keys(hdf5_file)

# Print the found keys
for key in all_keys:
    print(key)


'''

In [7]:
# import dataset

# importing electron dataset and seperating images and labels
#electron_dataset = h5py.File('SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5',"r")
electron_dataset = h5py.File('/content/drive/MyDrive/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5',"r")
electron_imgs=np.array(electron_dataset["X"])
electron_labels=np.array(electron_dataset["y"],dtype=np.int64)

# importing photon dataset and seperating images and labels
#photon_dataset = h5py.File('SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5',"r")
photon_dataset = h5py.File('/content/drive/MyDrive/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5',"r")
photon_imgs=np.array(photon_dataset["X"])
photon_labels=np.array(photon_dataset["y"],dtype=np.int64)

print("done")

done


In [8]:
print(electron_imgs.shape)
print(electron_imgs.dtype)
print(electron_labels)
print(photon_imgs.shape)
print(photon_labels)

(249000, 32, 32, 2)
float32
[1 1 1 ... 1 1 1]
(249000, 32, 32, 2)
[0 0 0 ... 0 0 0]


In [9]:
# concatenate electron and photon images/labels
img_arrs = torch.Tensor(np.vstack((photon_imgs,electron_imgs)))
labels = torch.Tensor(np.hstack((photon_labels,electron_labels))).to(torch.int64)
print(img_arrs.shape)
img_arrs = img_arrs.permute(0,3,1,2)
print(img_arrs.shape)

torch.Size([498000, 32, 32, 2])
torch.Size([498000, 2, 32, 32])


In [10]:
del electron_imgs,photon_imgs,electron_labels,photon_labels

In [11]:
from torch.utils.data import Dataset, DataLoader, random_split

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data  # data[:,:,4:-4,4:-4] #to cut external 4 zero padding
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {'image': self.data[idx], 'label': self.labels[idx]}
        return sample

In [12]:
# Assuming img_arrs and labels are already defined
custom_dataset = CustomDataset(img_arrs, labels)

# Split the dataset into training, validation, and test sets
train_size = int(0.7 * len(custom_dataset))
valid_size = int(0.2 * len(custom_dataset))
test_size = len(custom_dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(custom_dataset, [train_size, valid_size, test_size])
print(len(valid_dataset)+len(test_dataset)+len(train_dataset))

498000


transforms should be used here before data loader generation

In [13]:
# Create DataLoaders for training, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                           num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                           num_workers=NUM_WORKERS, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                                          num_workers=NUM_WORKERS, pin_memory=True)



Step 1 Tasks:

1- turn an image into patches

2- flatten the patch feature maps into a single dimension

3- Convert the output into Desried output (flattened 2D patches): (196, 768) -> N×(P2⋅C) #Current shape: (1, 768, 196)


In [14]:
# next(iter(...)) catches the first batch of the data loader
# If shuffle is True, this will return a different batch every time we run this cell
# For iterating over the whole dataset, we can simple use "for batch in data_loader: ..."
data_inputs = next(iter(train_loader))

# The shape of the outputs are [batch_size, d_1,...,d_N] where d_1,...,d_N are the
# dimensions of the data point returned from the dataset class
print("Data inputs", data_inputs['image'].shape, "\n")
# print("Full",data_inputs)

Data inputs torch.Size([256, 2, 32, 32]) 



# For Max pool:
output size = 1+ (input_size-kernel_size)/stride

In [ ]:
# pool=nn.MaxPool2d(kernel_size=4,stride=2)
# plt.imshow(data_inputs['image'][0,0:1,:,:].permute(1,2,0))
# plt.title('Electron Image ch1')
# plt.show()
# plt.imshow(pool(data_inputs['image'][0,0:1,:,:]).permute(1,2,0))
# plt.show()
# plt.imshow(pool(pool(data_inputs['image'][0,0:1,:,:])).permute(1,2,0))
# plt.show()
# print(pool(pool(data_inputs['image'][0,0:1,:,:])))

In [ ]:
'''
my_conv1 = nn.Conv2d(in_channels=2, out_channels=31, kernel_size=3, padding=1)
output=my_conv1((data_inputs['image']))
print(output.shape)
relu = nn.ReLU()
output=output.view(BATCH_SIZE,-1)
print(output.shape)
fc = nn.Linear(output.shape[1],2)
fc(output)
'''


In [16]:
class MyModel(nn.Module):

    def __init__(self, num_classes=1,dropout_rate=DROP_OUT):
        super().__init__()
        self.my_conv1 = nn.Conv2d(in_channels=2, out_channels=8, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
        #self.my_conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        #self.my_conv3 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.Linear1=nn.Linear(16*16*8,16*16*2)
        self.Linear2=nn.Linear(16*16*2,24*8)
        self.Linear3=nn.Linear(24*8,4)
        #self.Linear4=nn.Linear(4,2)
        # self.Linear1=nn.Linear(16*16*8,8*8*1)
        # self.Linear2=nn.Linear(8*8*1,24*8)
        # self.Linear3=nn.Linear(24*8,4)
        self.Linear=nn.Linear(4,1)

    def forward(self, x):
        x['image'] =self.my_conv1(x['image'])
        x['image'] =self.relu(x['image'])
        x['image'] =self.pool(x['image'])
        x['image'] =x['image'].view(BATCH_SIZE,-1)

        x['image'] =self.Linear1(x['image'])
        x['image'] =self.relu(x['image'])
        x['image'] =self.dropout(x['image'])
        x['image'] =self.Linear2(x['image'])
        x['image'] =self.relu(x['image'])
        x['image'] =self.dropout(x['image'])
        x['image'] =self.Linear3(x['image'])
        x['image'] =self.relu(x['image'])
        x['image'] =self.Linear(x['image'])
        return x['image']



In [17]:
import torch.nn as nn
class My2Model(nn.Module):

    def __init__(self, num_classes=1, dropout_rate=0.2):
        super().__init__()

        self.conv1=nn.Conv2d(in_channels=1,out_channels=4,kernel_size=3,stride=1,padding=1)
        #self.conv2=nn.Conv2d(in_channels=4,out_channels=8,kernel_size=3,stride=1,padding=3)
        #self.conv3=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1)
        #self.conv4=nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,stride=1,padding=1)
        #self.conv5=nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,stride=1,padding=1)
        #self.bn1=nn.BatchNorm2d(16)
        #self.bn2=nn.BatchNorm2d(64)
        #self.bn2=nn.BatchNorm2d(32)
        #self.b3=nn.BatchNorm2d(16)
        self.pool=nn.MaxPool2d(kernel_size=2,stride=2)
        self.unfold = nn.Unfold(kernel_size=3, stride=1, padding=1)
        self.sparse=nn.Conv2d(in_channels=3*3,out_channels=2,kernel_size=1)

        self.relu=nn.ReLU()
        self.dropout=nn.Dropout(0.1)
        #self.fc0=nn.Linear(12*4*2,12*4) #
        self.fc1=nn.Linear(12*4*2,32)
        self.fc2=nn.Linear(32,16)
        self.fc3=nn.Linear(16,8)
        self.fc4=nn.Linear(8,4)
        self.fc5=nn.Linear(4,2)
        self.out=nn.Linear(2,1)


    def forward(self, x):
        channel1 = x['image'][:, 0:1, :, :]  # Extract first channel
        channel2 = x['image'][:, 1:2, :, :]  # Extract second channel
        #print(channel1.shape)
        convchannel1 = self.conv1(channel1)
        convchannel2 = self.conv1(channel2)
        #print(convchannel1.shape)

        '''
        convchannel1 = convchannel1.view(convchannel1.size(0), -1)
        convchannel2 = convchannel2.view(convchannel2.size(0), -1)
        convchannel = torch.cat([convchannel1, convchannel2], dim=1)
        '''

        # Compute normalized histograms for each output channel of the convolutional layer
        hist_conv = []
        for batch in  convchannel1:# data_inputs['image']:
            batch_hist = []
            for channel in batch:
                # Create a mask of non-zero elements
                mask = channel != 0
                # Filter the tensor using the mask
                #non_zero_elements = channel[mask]
                non_zero_elements = channel[mask]*channel[mask]
                #min_val=non_zero_elements.min()
                max_val=non_zero_elements.max()
                # Perform min-max normalization to scale values between 0 and 1
                #normalized_tensor = 2 * (non_zero_elements - min_val) / (max_val - min_val) - 1
                normalized_tensor = non_zero_elements/max_val
                # Compute histogram
                # Set all elements within the tensor to zero
                hist = torch.histc(normalized_tensor, bins=12, min=0, max=+1)
                # Normalize the histogram and append to the list
                batch_hist.append(hist)

            # Concatenate the histograms for all channels in the batch
            batch_hist = torch.cat(batch_hist, dim=0)
            hist_conv.append(batch_hist)

        # Stack the histograms along a new dimension to get the desired tensor shape
        histograms_tensor1 = torch.stack(hist_conv, dim=0).to(device='cuda')
        # Shape will be (batch_size, channel_no * bins)
        #print("MyModel")
        #print(histograms_tensor1.shape)
        # hist_ch1 = self.fc0(histograms_tensor)
        # print('hello')
        # print(hist_ch1.shape)





        # Compute normalized histograms for each output channel of the convolutional layer
        hist_conv = []
        for batch in  convchannel2:# data_inputs['image']:
            batch_hist = []
            for channel in batch:
                # Create a mask of non-zero elements
                mask = channel != 0
                # Filter the tensor using the mask
                #non_zero_elements = channel[mask]
                non_zero_elements = channel[mask]*channel[mask]
                #min_val=non_zero_elements.min()
                max_val=non_zero_elements.max()
                # Perform min-max normalization to scale values between 0 and 1
                #normalized_tensor = 2 * (non_zero_elements - min_val) / (max_val - min_val) - 1
                normalized_tensor = non_zero_elements/max_val
                # Compute histogram
                # Set all elements within the tensor to zero
                hist = torch.histc(normalized_tensor, bins=12, min=0, max=+1)
                # Normalize the histogram and append to the list
                batch_hist.append(hist)

            # Concatenate the histograms for all channels in the batch
            batch_hist = torch.cat(batch_hist, dim=0)
            hist_conv.append(batch_hist)

        # Stack the histograms along a new dimension to get the desired tensor shape
        histograms_tensor2 = torch.stack(hist_conv, dim=0).to(device='cuda')
        # Shape will be (batch_size, channel_no * bins)
        #print("MyModel")
        #print(histograms_tensor2.shape)
        # hist_ch2 =self.fc0(histograms_tensor)
        # print('hello')
        # print(hist_ch2.shape)


        convchannel = torch.cat([histograms_tensor1, histograms_tensor2], dim=1)
        #print(convchannel.shape)

        convchannel = self.fc1(convchannel)
        convchannel = self.relu(convchannel)

        convchannel = self.fc2(convchannel)
        convchannel = self.relu(convchannel)

        convchannel = self.fc3(convchannel)
        convchannel = self.relu(convchannel)

        convchannel = self.fc4(convchannel)
        convchannel = self.relu(convchannel)

        convchannel = self.fc5(convchannel)
        convchannel = self.relu(convchannel)


        convchannel = self.out(convchannel)

        return convchannel


In [18]:
model = My2Model()
model.to(device)
print(model)

My2Model(
  (conv1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (unfold): Unfold(kernel_size=3, dilation=1, padding=1, stride=1)
  (sparse): Conv2d(9, 2, kernel_size=(1, 1), stride=(1, 1))
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=96, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
  (out): Linear(in_features=2, out_features=1, bias=True)
)


In [ ]:
# clearing cuda cache memory
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [21]:
def eval_model(model, data_loader):
    model.eval() # Set model to eval mode
    true_preds, num_preds = 0., 0.

    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs in data_loader:
            is_last_batch = data_inputs['image'].shape[0] < BATCH_SIZE
            if is_last_batch:
                break

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs['image'] = data_inputs['image'].to(device)
            data_inputs['label'] = data_inputs['label'].to(device)
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1)
            preds = torch.sigmoid(preds) # Sigmoid to map predictions between 0 and 1
            pred_labels = (preds >= 0.5).long() # Binarize predictions to 0 and 1

            # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
            true_preds += (pred_labels == data_inputs['label']).sum()
            num_preds += BATCH_SIZE

    acc = true_preds / num_preds
    print(f"Accuracy of the model: {100.0*acc:4.2f}%")
    return acc

In [23]:
# Input to the optimizer are the parameters of the model: model.parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

loss_module = nn.BCEWithLogitsLoss()


def train_model(model, optimizer, data_loader, val_loader,loss_module, num_epochs=EPOCHS):
    train_losses = []  # Changed variable name from train_loss to train_losses
    val_losses = []

    # Set model to train mode
    model.train()
    running_loss = 0.0
    # Training loop
    for epoch in tqdm(range(num_epochs)):
        for data_inputs in data_loader:
            # Check if the current batch is the last one
            is_last_batch = data_inputs['image'].shape[0] < BATCH_SIZE
            if is_last_batch:
                break

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs['image'] = data_inputs['image'].to(device)
            data_inputs['label'] = data_inputs['label'].to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)
            #print(preds.shape)
            # Ensure that the predictions have the same data type as the labels
            preds = preds.squeeze(dim=1).to(data_inputs['label'].float())

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_inputs['label'].float())

            ## Step 4: Perform backpropagation
            optimizer.zero_grad()
            loss.backward()

            ## Step 5: Update the parameters
            optimizer.step()

            running_loss += loss.item() * BATCH_SIZE # batch_size

        train_loss = running_loss / len(train_loader.dataset)
        train_losses.append(train_loss)

        #validation plase
        model.eval()
        running_loss = 0.0
        with torch.no_grad():
          for data_inputs in val_loader:
            is_last_batch = data_inputs['image'].shape[0] < BATCH_SIZE
            if is_last_batch:
                break

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs['image'] = data_inputs['image'].to(device)
            data_inputs['label'] = data_inputs['label'].to(device)

            ## Step 2: Run the model on the input data
            preds = model(data_inputs)

            # Ensure that the predictions have the same data type as the labels
            preds = preds.squeeze(dim=1).to(data_inputs['label'].float())

            ## Step 3: Calculate the loss
            loss = loss_module(preds, data_inputs['label'].float())
            running_loss += loss.item() * BATCH_SIZE
        val_loss = running_loss / len(val_loader.dataset)
        val_losses.append(val_loss)

        print(f"Epoch {epoch+1}/{num_epochs} - Train loss {train_loss}, Validation Loss {val_loss}")
        eval_model(model, test_loader)

In [ ]:
train_model(model, optimizer, train_loader, val_loader, loss_module)

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch 1/40 - Train loss 0.6759177661749117, Validation Loss 0.6706020102443465


  2%|▎         | 1/40 [14:58<9:44:06, 898.62s/it]

Accuracy of the model: 59.48%
Epoch 2/40 - Train loss 0.8585479814679337, Validation Loss 0.6664018801034215


  5%|▌         | 2/40 [29:46<9:25:08, 892.33s/it]

Accuracy of the model: 59.69%
Epoch 3/40 - Train loss 0.8552128612017126, Validation Loss 0.668077109769645


  8%|▊         | 3/40 [44:29<9:07:42, 888.19s/it]

Accuracy of the model: 59.87%
Epoch 4/40 - Train loss 0.8545491718110584, Validation Loss 0.6647989953282368


 10%|█         | 4/40 [59:10<8:51:03, 885.10s/it]

Accuracy of the model: 59.96%


In [ ]:
'''
def eval_model(model, data_loader):
    model.eval() # Set model to eval mode
    true_preds, num_preds = 0., 0.

    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs in data_loader:
            is_last_batch = data_inputs['image'].shape[0] < BATCH_SIZE
            if is_last_batch:
                break

            ## Step 1: Move input data to device (only strictly necessary if we use GPU)
            data_inputs['image'] = data_inputs['image'].to(device)
            data_inputs['label'] = data_inputs['label'].to(device)
            preds = model(data_inputs)
            preds = preds.squeeze(dim=1)
            preds = torch.sigmoid(preds) # Sigmoid to map predictions between 0 and 1
            pred_labels = (preds >= 0.5).long() # Binarize predictions to 0 and 1

            # Keep records of predictions for the accuracy metric (true_preds=TP+TN, num_preds=TP+TN+FP+FN)
            true_preds += (pred_labels == data_inputs['label']).sum()
            num_preds += BATCH_SIZE

    acc = true_preds / num_preds
    return acc
    '''

In [ ]:
#eval_model(model, test_loader)

tensor(0.7020, device='cuda:0')